# MOwNiT - Laboratorium 2 - Dominik Jędraszek

#### Ponieżej programy do obliczania iloczynu skalarnego wektorów (o) i mnożenia macierzy kwadratowej przez wektor (m).  Dodatkowo funkcje weryfikują poprawność podawanych parametrów oraz zapisują wyniki do pliku csv.

In [2]:
# using Pkg
#Pkg.add("LinearAlgebra")
# Pkg.add("DataFrames")
#Pkg.add("CSV")
using LinearAlgebra
using DataFrames 
using CSV

 #iloczyn skalarny
function o(a,b)
    if(length(a)!=length(b)) return error("Invalid vectors")
    else
        t=@elapsed res= dot(a,b)
        df = DataFrame(operation="iloczyn skalarny",size=length(a),time=t)
        CSV.write("res.csv", df,header =  filesize("res.csv")==0, append = true)
        return res
    end
end

#mnozenie macierzy przez wektor
function m(A,s)
    if(size(A,1)^2 != length(A)) return error("Invalid matrix")
    else
        u = UniformScaling(s);
        t=@elapsed res= A*u
        df = DataFrame(operation="mnozenie macierzy",size=size(A,1),time=t)
        CSV.write("res.csv", df,header =  filesize("res.csv")==0, append = true)
        return res
    end
end

m (generic function with 1 method)

#### Test działania funkcji z wykorzystaniem losowych wektorów (rand) o różnych wielkościach. Wyniki czasu zapisane do pliku res.csv.

In [12]:
for _ in 1:10
    for i in 1:10
        randVector1=rand((1: 100), i*i,1)
         randVector2=rand((1: 100), i*i,1)
        o(randVector1,randVector1)
    end

    for i in 1:10
        randMatrix=rand((1: 100), i*i,i*i)
        randVector=rand((1:100))
        m(randMatrix,randVector)
    end
end

#### Wczytujemy dane. Struktura kolumn pliku csv jest następująca: operation (wykonywana operacja),size (wielkość danych (wektora/macierzy)),time (czas wykonywania operacji)

In [3]:
mydata = CSV.read("res.csv", delim=",", DataFrame)

,operation,size,time
,String31,Int64,Float64
1,iloczyn skalarny,1,1.83e-7
2,iloczyn skalarny,4,9.6e-8
3,iloczyn skalarny,9,8.5e-8
4,iloczyn skalarny,16,1.37e-7
5,iloczyn skalarny,25,1.26e-7
6,iloczyn skalarny,36,1.25e-7
7,iloczyn skalarny,49,1.41e-7
8,iloczyn skalarny,64,1.05e-7
9,iloczyn skalarny,81,1.22e-7


#### Przystosowywujemy tabelę do rysowania wykresów średnich czasów obliczenia operacji w zależności od rozmiaru wektora wraz z błędem. W tym celu: 
- grupujemy testy o tych samych operacjach oraz wielkościach,
- dodajemy i obliczamy kolumny: średniej (time_mean) i odchylenia standardowego(time_std).

In [4]:
using Statistics
grouped = sort(combine(groupby(mydata, [:operation,:size]), nrow=> :liczba_testow,"time" => mean,"time"=>std),[:operation,:size])

,operation,size,liczba_testow,time_mean,time_std
,String31,Int64,Int64,Float64,Float64
1,iloczyn skalarny,1,10,7.44e-8,3.84973e-8
2,iloczyn skalarny,4,10,1.007e-7,2.54124e-8
3,iloczyn skalarny,9,10,1.067e-7,2.09658e-8
4,iloczyn skalarny,16,10,1.09e-7,1.92988e-8
5,iloczyn skalarny,25,10,1.141e-7,1.98631e-8
6,iloczyn skalarny,36,10,1.406e-7,4.10073e-8
7,iloczyn skalarny,49,10,1.291e-7,1.58916e-8
8,iloczyn skalarny,64,10,1.329e-7,1.79843e-8
9,iloczyn skalarny,81,10,1.547e-7,3.46604e-8


#### Tworzymy wykresy wykorzystając przy tym właściwość layout. Dodatkowo łączymy odpowiednie wykresy, by przedstawić odchylenie standardowe czasu obliczeń oraz średnia na jednym wykresie.

In [ ]:
using Plots
p1 =scatter(grouped.size, grouped.time_mean, group=grouped.operation,  colour = :yellow, layout=2,title=["Iloczyn skalarny" "Mnożenie macierzy" ],label="średnia")
p2 =plot!(p1,grouped.size, grouped.time_mean+grouped.time_std, group=grouped.operation,  colour = :red, layout=2,label="błąd")
plot!(p1,grouped.size, grouped.time_mean-grouped.time_std, group=grouped.operation,  colour = :red, layout=2,label="")
xlabel!("Rozmiar wektora")
ylabel!("Czas")

WNIOSKI:
- czas wykonywania obydwu funkcji jest proporcjonalny do wielkości wektora.
- iloczyn skalarny jest wykonywany przez komputer szybciej, niż mnożenie macierzy.
- funkcja do obliczania iloczynu skalarnego jest stabilniejsza.
